### Classificação com Bert

<a href="https://www.tensorflow.org/text/tutorials/classify_text_with_bert">Tutorial link</a>

In this notebook, you will:

- Load the IMDB dataset
- Load a BERT model from TensorFlow Hub
- Build your own model by combining BERT with a classifier
- Train your own model, fine-tuning BERT as part of that
- Save your model and use it to classify sentences
If you're new to working with the IMDB dataset, please see Basic text classification for more details.

#### Setup

In [ ]:
# A dependency of the preprocessing for BERT inputs
! pip install -q -U "tensorflow-text==2.8.*"

In [6]:
! pip install -q tf-models-official==2.7.0

  ERROR: Command errored out with exit status 1:
   command: /home/kaline/anaconda3/bin/python /home/kaline/anaconda3/lib/python3.7/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /tmp/tmpl4u35vbu
       cwd: /tmp/pip-install-fjsc0lls/pycocotools
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-cpython-37
  creating build/lib.linux-x86_64-cpython-37/pycocotools
  copying pycocotools/coco.py -> build/lib.linux-x86_64-cpython-37/pycocotools
  copying pycocotools/__init__.py -> build/lib.linux-x86_64-cpython-37/pycocotools
  copying pycocotools/cocoeval.py -> build/lib.linux-x86_64-cpython-37/pycocotools
  copying pycocotools/mask.py -> build/lib.linux-x86_64-cpython-37/pycocotools
  running build_ext
  cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
  /tmp/pip-build-env-kmvpd3eg/overlay/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'langua

In [7]:
import os
import shutil
import tensorflow as tf

tf.get_logger().setLevel('ERROR')

#### Análise de sentimentos

Este notebook treina um modelo de análise de sentimentos para classificar avaliações de filmes como positivas ou negativas, baseadas no texto da revisão.
Você vai usar <a href="https://ai.stanford.edu/~amaas/data/sentiment/">Large Movie Review Dataset</a>, no qual contém o texto de 50,000 avaliações de filmes do <a href="https://www.imdb.com/">Internet Movie database</a>

#### Download o IMDB dataset

Baixar o dataset e explorar a estrutura do diretório.

In [12]:


url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url, untar=True, cache_dir='.', cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove pastas não utilizadas para ser mais fácil carregar os dados
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Na próxima, você vai utilizar a utilidade text_dataset_from_directory para criar o rotulado tf.data.Dataset

O IMDB dataset já foi dividido em treino e teste, mas necessita um conjunto de validação. Vamos criar um conjunto de validação usando a divisão 80:20 do dados de treinamento usando o parâmetro validation_split abaixo.

Obs: necessário especificar um seed ou shuffle=False, assim o conjunto de validação e treinamento não terão sobreposição (overlap).

In [13]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)
    


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
